# Dev notebook for manual work

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Creating ptr.csv

In [28]:
# read excel file in data
data = pd.read_excel('../../data/Additional file 2.xlsx', sheet_name='PTR_Eraslan2019')

In [5]:
# Store
data.to_csv('../../data/ptr.csv', index=False)

In [4]:
data.head(10)

,GeneName,EnsemblGeneID,EnsemblTranscriptID,EnsemblProteinID,Adrenal,Appendices,Brain,Colon,Duodenum,Uterus,...,Rectum,Salivarygland,Smallintestine,Smoothmuscle,Spleen,Stomach,Testis,Thyroid,Tonsil,Urinarybladder
0,A1BG,ENSG00000121410,ENST00000263100,ENSP00000263100,NaN,8.277,NaN,NaN,NaN,NaN,...,NaN,7.718,NaN,NaN,7.313,NaN,NaN,NaN,NaN,NaN
1,A1CF,ENSG00000148584,ENST00000373993,ENSP00000363105,NaN,NaN,NaN,5.135,5.371,NaN,...,5.147,NaN,5.202,NaN,NaN,5.8143,NaN,NaN,NaN,NaN
2,A2M,ENSG00000175899,ENST00000318602,ENSP00000323929,6.290,6.328,5.948,5.811,6.068,5.383,...,6.081,5.726,5.699,4.997,5.136,6.5349,5.820,6.060,5.675,5.8286
3,A2ML1,ENSG00000166535,ENST00000299698,ENSP00000299698,NaN,NaN,3.995,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.350,NaN,5.249,NaN
4,A4GALT,ENSG00000128274,ENST00000401850,ENSP00000384794,3.843,4.601,NaN,NaN,NaN,NaN,...,4.731,4.508,NaN,NaN,NaN,4.0613,4.832,NaN,NaN,4.2430
5,A4GNT,ENSG00000118017,ENST00000236709,ENSP00000236709,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.3992,NaN,NaN,NaN,NaN
6,AAAS,ENSG00000094914,ENST00000209873,ENSP00000209873,4.513,4.434,4.461,4.536,4.917,4.677,...,4.440,4.350,4.120,4.241,3.975,5.0890,4.991,4.651,4.505,4.4699
7,AACS,ENSG00000081760,ENST00000316519,ENSP00000324842,4.559,5.015,5.120,4.426,5.285,4.640,...,4.817,4.807,4.395,4.310,NaN,4.7653,4.552,4.588,4.901,4.8409
8,AADAC,ENSG00000114771,ENST00000232892,ENSP00000232892,5.110,NaN,NaN,NaN,5.008,NaN,...,NaN,NaN,5.076,NaN,NaN,4.7098,NaN,NaN,NaN,NaN
9,AADAT,ENSG00000109576,ENST00000515480,ENSP00000423341,5.227,NaN,4.165,4.115,NaN,4.801,...,4.282,NaN,NaN,NaN,NaN,4.5203,3.657,3.563,NaN,4.2271


In [52]:
# Total available annotated data with PTRs
data[data.columns[4:]].count().sum()

232919

## Loading dev data file

In [78]:
# load data.pkl
import pickle

with open('../../data/ptr_data.pkl', 'rb') as f:
    data = pickle.load(f)
    
# Insights:
# - data is a dictionary with keys as gene ids
# - targets are tissue specific PTRs
# - targets_bin 1 if "up" (high-PTR), 0 if "down" (low-PTR)

In [9]:
len(data.keys())

11279

In [8]:
data["ENST00000012049"].keys()

dict_keys(['fasta', 'fasta_ohe', 'bed', 'bed_annotation', 'codon_freq', 'targets', 'targets_bin'])

In [11]:
data["ENST00000012049"]

{'fasta': 'AATCCGTGGTCTGGTACAGGTTTCAGGGCAAAGCGGCCATGCGTTCCGGGGGCCGCGGGCGACCCCGCCTGCGGCTGGGGGAACGTGGCCTCATGGAGCCACTCTTGCCGCCGAAGCGCCGCCTGCTACCGCGGGTTCGGCTCTTGCCTCTGTTGCTGGCGCTGGCCGTGGGCTCGGCGTTCTACACCATTTGGAGCGGCTGGCACCGCAGGACTGAGGAGCTGCCGCTGGGCCGGGAGCTGCGGGTCCCATTGATCGGAAGCCTCCCCGAAGCCCGGCTGCGGAGGGTGGTGGGACAACTGGATCCACAGCGTCTCTGGAGCACTTATCTGCGCCCCCTGCTGGTTGTGCGAACCCCGGGCAGCCCGGGAAATCTCCAAGTCAGAAAGTTCCTGGAGGCCACGCTGCGGTCCCTGACAGCAGGTTGGCACGTGGAGCTGGATCCCTTCACAGCCTCAACACCCCTGGGGCCAGTGGACTTTGGCAATGTGGTGGCCACACTGGACCCAAGGGCTGCCCGTCACCTCACCCTTGCCTGCCATTATGACTCGAAGCTCTTCCCACCCGGATCGACCCCCTTTGTAGGGGCCACGGATTCGGCTGTGCCCTGTGCCCTGCTGCTGGAGCTGGCCCAAGCACTTGACCTGGAGCTGAGCAGGGCCAAAAAACAGGCAGCCCCGGTGACCCTGCAACTGCTCTTCTTGGATGGTGAAGAGGCGCTGAAGGAGTGGGGACCCAAGGACTCCCTTTACGGTTCCCGGCACCTGGCCCAGCTCATGGAGTCTATACCTCACAGCCCCGGCCCCACCAGGATCCAGGCTATTGAGCTCTTTATGCTTCTTGATCTCCTGGGAGCCCCCAATCCCACCTTCTACAGCCACTTCCCTCGCACGGTCCGCTGGTTCCATCGGCTGAGGAGCATTGAGAAGCGTCTGCACCGTTTGAACCTGCTGCAGTCTCATCCCCAGGAAGTGATGTACTTCCAACCC

In [14]:
len(data["ENST00000012049"]["bed_annotation"])

2125

In [13]:
len(data["ENST00000012049"]["fasta_ohe"])

2125

## Kidney toy example

CV-5 in paper:

mean accuracy: 0.7314

mean ROC:      0.7896

In [79]:
df = pd.DataFrame(data).T

In [83]:
df.head()

,fasta,fasta_ohe,bed,bed_annotation,codon_freq,targets,targets_bin
ENST00000263100,ATTGCTGCAGACGCTCACCCCAGACACTCACTGCACCGGAGTGAGC...,"[0, 3, 3, 2, 1, 3, 2, 1, 0, 2, 0, 1, 2, 1, 3, ...","[[ENST00000263100.8, 0, 55, 5UTR, ., +], [ENST...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[0.006048387096774193, 0.0020161290322580645, ...","[nan, 8.277, nan, nan, nan, nan, nan, 7.841, n...","[nan, 1.0, nan, nan, nan, nan, nan, nan, nan, ..."
ENST00000373993,ATAATCAAGGAAACCTTTTCCGGGTGGGGATCTCTGAAATTACTCA...,"[0, 3, 0, 0, 3, 1, 0, 0, 2, 2, 0, 0, 0, 1, 1, ...","[[ENST00000373993.6, 0, 92, 5UTR, ., +], [ENST...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[0.04201680672268908, 0.015126050420168067, 0....","[nan, nan, nan, 5.135, 5.371, nan, nan, nan, n...","[nan, nan, nan, 0.0, nan, nan, nan, nan, nan, ..."
ENST00000318602,GGGACCAGATGGATTGTAGGGAGTAGGGTACAATACAGTCTGTTCT...,"[2, 2, 2, 0, 1, 1, 0, 2, 0, 3, 2, 2, 0, 3, 3, ...","[[ENST00000318602.12, 0, 70, 5UTR, ., +], [ENS...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[0.02305084745762712, 0.014915254237288136, 0....","[6.29, 6.328, 5.948, 5.811, 6.068, 5.383, 5.88...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
ENST00000299698,GACCCTGGAAAAATCTGTCTCACCCACAAAGATGTGGGCTCAGCTC...,"[2, 0, 1, 1, 1, 3, 2, 2, 0, 0, 0, 0, 0, 3, 1, ...","[[ENST00000299698.12, 0, 31, 5UTR, ., +], [ENS...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[0.019243986254295534, 0.01443298969072165, 0....","[nan, nan, 3.995, nan, nan, nan, 4.129, nan, n...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
ENST00000401850,TGCACTTCTGTGCCTCAATTTCCTCATCTGTAGGGTGGGGGTGGTG...,"[3, 2, 1, 0, 1, 3, 3, 1, 3, 2, 3, 2, 1, 1, 3, ...","[[ENST00000401850.5, 0, 490, 5UTR, ., +], [ENS...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[0.00847457627118644, 0.0, 0.00282485875706214...","[3.843, 4.601, nan, nan, nan, nan, 4.013, 3.68...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [86]:
# extract only first element of targets_bin
df["targets_bin_kidney"] = df["targets_bin"].apply(lambda x: x[11])

In [91]:
df_kidney = df[["codon_freq", 'targets_bin_kidney']]
df_kidney = df_kidney.dropna(subset=["targets_bin_kidney"])
df_kidney

(848, 2)

In [94]:
# Store kidney example
df_kidney.to_pickle('../../data/kidney_example.pkl')

In [31]:
# CHECKPOINT
# Load kidney example
# df_kidney = pd.read_csv('../../data/kidney_example.csv', index_col=0)
df_kidney = pd.read_pickle('../../data/kidney_example.pkl')

In [33]:
x = np.array(df_kidney["codon_freq"].tolist())
y = np.array(df_kidney["targets_bin_kidney"].tolist())

In [27]:
# Try MLP
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate

clf = MLPClassifier(random_state=42, max_iter=1000)
cv_scores = cross_validate(clf, x, y, cv=5, scoring=['accuracy', 'roc_auc'], return_train_score=True)
cv_scores

C:\Users\Felix\miniforge3\envs\master-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


{'fit_time': array([1.71376824, 1.73701215, 1.57801628, 2.43115544, 2.06366014]),
 'score_time': array([0.00299931, 0.00300193, 0.0030005 , 0.00300264, 0.00200272]),
 'test_accuracy': array([0.68823529, 0.67647059, 0.74117647, 0.71597633, 0.71005917]),
 'train_accuracy': array([0.78171091, 0.76696165, 0.75663717, 0.78939617, 0.77466863]),
 'test_roc_auc': array([0.75692137, 0.75653979, 0.79128028, 0.79565826, 0.7662465 ]),
 'train_roc_auc': array([0.86364427, 0.84943916, 0.85216287, 0.8761561 , 0.86674244])}

In [28]:
# mean and std of accuracy
print(f"Mean accuracy: {cv_scores['test_accuracy'].mean()}")
print(f"Std accuracy: {cv_scores['test_accuracy'].std()}")

Mean accuracy: 0.7063835711799513
Std accuracy: 0.022539846692171767


In [29]:
# mean and std of ROC
print(f"Mean ROC: {cv_scores['test_roc_auc'].mean()}")
print(f"Std ROC: {cv_scores['test_roc_auc'].std()}")

Mean ROC: 0.7733292408618715
Std ROC: 0.01686471982780332


In [22]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=42)
cv_scores = cross_validate(clf, x, y, cv=5, scoring=['accuracy', 'roc_auc'], return_train_score=True)
cv_scores

{'fit_time': array([0.17111826, 0.20527482, 0.23377776, 0.19441032, 0.17647743]),
 'score_time': array([0.01018357, 0.01899767, 0.01103258, 0.01151872, 0.0120008 ]),
 'test_accuracy': array([0.68823529, 0.72352941, 0.71764706, 0.75739645, 0.72189349]),
 'train_accuracy': array([0.75663717, 0.74926254, 0.74926254, 0.73932253, 0.74963181]),
 'test_roc_auc': array([0.76785714, 0.77605536, 0.78242215, 0.81568627, 0.79187675]),
 'train_roc_auc': array([0.83018187, 0.82743197, 0.83093885, 0.82155686, 0.82829391])}

In [24]:
# mean and std of accuracy
print(f"Mean accuracy: {cv_scores['test_accuracy'].mean()}")
print(f"Std accuracy: {cv_scores['test_accuracy'].std()}")

Mean accuracy: 0.7217403411068569
Std accuracy: 0.021972371026578615


In [26]:
# mean and std of ROC
print(f"Mean ROC: {cv_scores['test_roc_auc'].mean()}")
print(f"Std ROC: {cv_scores['test_roc_auc'].std()}")

Mean ROC: 0.7867795353435492
Std ROC: 0.016453770536395674
